In [32]:
sports_news_text = {'title':'Sports Section',
                    'text':"The San Francisco 49ers are heading to the super bowl in a football showdown!"}

In [33]:
finance_news_text = {'title':"Finance Section",
                     'text':"Meta stock has reached all time highs and has become a major part of the S&P500."}

In [34]:
import google.generativeai as genai
genai.configure(api_key=api_key)

In [35]:
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004
models/aqa


In [36]:
sports_embedding_vector = genai.embed_content(model='models/embedding-001', content=sports_news_text['text'],task_type='retrieval_document')

In [37]:
type(sports_embedding_vector['embedding'])

list

In [38]:
#sports_embedding_vector['embedding']
len(sports_embedding_vector['embedding'])
#sports_embedding_vector.keys()

768

In [39]:
finance_embedding_vector = genai.embed_content(model='models/embedding-001', content=finance_news_text['text'],task_type='retrieval_document')
len(finance_embedding_vector['embedding'])
finance_embedding_vector.keys()

dict_keys(['embedding'])

In [40]:
def embed_text(text):
    return genai.embed_content(model='models/embedding-001',content=text,
                             task_type='retrieval_document')['embedding']


In [41]:
import pandas as pd #chromaDb pinecone


In [42]:
#documents
df = pd.DataFrame()

In [43]:
documents = [finance_news_text,sports_news_text]
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Finance Section,Meta stock has reached all time highs and has ...
1,Sports Section,The San Francisco 49ers are heading to the sup...


In [44]:
df['Embeddings'] = df['Text'].apply(embed_text)

In [45]:
df

,Title,Text,Embeddings
0,Finance Section,Meta stock has reached all time highs and has ...,"[0.055816136, -0.0018495269, -0.024931083, 0.0..."
1,Sports Section,The San Francisco 49ers are heading to the sup...,"[0.013654178, -0.010523142, -0.053972486, -0.0..."


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       2 non-null      object
 1   Text        2 non-null      object
 2   Embeddings  2 non-null      object
dtypes: object(3)
memory usage: 180.0+ bytes


In [47]:
import numpy as np

def query_similarity_score(query, vector):
    # query --> "tell me about football sports?" TEXT STR
    # Vector --> Array (list) 
    
    #Query STR -->
    query_embedding = embed_text(query)
    
    #SIMILARITY
    return np.dot(query_embedding,vector)

In [48]:
#query = "Any interesting news about Meta stock performarce today?"
query = "Any interesting news about Meta stock performarce today?"

In [49]:
#def most_similar_document(query):
#df['Similarity'] = df['Embeddings'].apply(lambda vector: query_similarity_score(query,vector))

In [50]:
df['Similarity'] = df['Embeddings'].apply(lambda vector: query_similarity_score(query,vector))

In [51]:
df.sort_values('Similarity',ascending=False)[['Title','Text']].iloc[0]

Title                                      Finance Section
Text     Meta stock has reached all time highs and has ...
Name: 0, dtype: object

In [52]:
def most_similar_document(query):
    df['Similarity'] = df['Embeddings'].apply(lambda vector: query_similarity_score(query,vector))
    title = df.sort_values('Similarity',ascending=False)[['Title','Text']].iloc[0]['Title']
    text = df.sort_values('Similarity',ascending=False)[['Title','Text']].iloc[0]['Text']
    return title,text

In [53]:
def RAG(query):
    title,text = most_similar_document(query)
    model = genai.GenerativeModel('gemini-pro')
    prompt = f'Answer this query:\n{query}.\nOnly use this context to answer:\n{text}'
    response = model.generate_content(prompt)
    return f'{response.text}\n\nSource Document:{title}'

In [54]:
#print(RAG("Do you have any information about the upconming football?"))
print(RAG("How is the world of stocks today? something important?"))

The provided context does not contain information about the overall state of the stock market today, so I cannot answer the query.

Source Document:Finance Section


In [55]:
print(RAG("Anything interesting happening in the world of sports?"))

The San Francisco 49ers are heading to the super bowl in a football showdown.

Source Document:Sports Section


In [56]:
import os
import pandas as pd
from PyPDF2 import PdfReader

df = pd.DataFrame(columns=['Title', 'Text'])

for file_name in os.listdir('.'):
    if file_name.endswith('.pdf'):
        try:
            # Open the PDF file
            with open(file_name, 'rb') as file:
                # Initialize a PDF file reader
                pdf_reader = PdfReader(file)
                # Initialize text variable to store the content of the PDF
                text = ''
                # Iterate through each page in the PDF
                for page_num in range(len(pdf_reader.pages)):
                    # Extract text from the page
                    text += pdf_reader.pages[page_num].extract_text()
                    text = text.replace('\n',' ')
                # Create a new DataFrame with the file's title and text
                new_row = pd.DataFrame({'Title': [file_name], 'Text': [text]})
                # Concatenate the new DataFrame row to the existing DataFrame
                df = pd.concat([df, new_row], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

In [57]:
print(df['Text'].iloc[0])

 Curso de C++. Capítulo 000 Introducción Bien, aquellos que hayáis seguido el curso desde sus comienzos, en septiembre de 2000,  conocéis la trayectoria y la evolución que ha tenido. El curso está ya muy avanzado, parecía imposible al principio, pero ya están tratados la mayor parte de los temas sobre C++. Lo que queda de comentar sobre C++ se reduce a un único tema: asm, y algo sobre el  modificador explicit. Actualmente estoy haciendo un repaso a fondo y añadiendo más ejercicios y ejemplos. Sigo esperando que este curso anime a los nuevos y futuros programadores autodidactas  a incorporarse a esta gran y potente herramienta que es el C++, ese era el objetivo original y sigo manteniéndolo. No he pretendido ser original, (al menos no demasiado), como dije que haría, he  consultado libros, tutoriales, revistas, listas de correo, news, páginas web... En fin, cualquier fuente de datos que he podido, con el fin de conseguir un buen nivel. Espero haber conseguido mi objetivo, y seguiré comp